# OpenAIRE Beginners Kit

The **OpenAIRE Graph** is an Open Access dataset containing metadata about research products (literature, datasets, software, and other research products) linked to other entities of the research ecosystem, such as organisations, grants, and data sources.

The large size of the OpenAIRE Graph is a major impediment for beginners to familiarise with the underlying data model and explore its contents. Working with the Graph in its full size typically requires access to a huge distributed computing infrastructure which cannot be easily accessible to everyone.

The OpenAIRE Beginner’s Kit aims to address this issue. It consists of two components: a subset of the Graph composed of the research products published between `2023-06-30` and `2024-02-29`, all the entities connected to them and the respective relationships, and the present Jupyter notebook that demonstrates how you can use `PySpark` to analyse the Graph and get answers to some interesting research questions.

## Import data

In [ ]:
import os
import requests
from urllib.parse import urlsplit
import tarfile
from pathlib import Path

zenodo_url = "https://zenodo.org/record/10837214/files/"

openaire_files = [zenodo_url + "communities_infrastructures.tar",
                  zenodo_url + "dataset.tar",
                  zenodo_url + "datasource.tar",
                  zenodo_url + "organization.tar",
                  zenodo_url + "otherresearchproduct.tar",
                  zenodo_url + "project.tar",
                  zenodo_url + "publication.tar",
                  zenodo_url + "relation.tar",
                  zenodo_url + "software.tar"]



def download_and_extract(url, path):
    tar_name = urlsplit(url).path.split('/')[-1] # publication.tar
    tar_path = os.path.join(path, tar_name) # data/raw/publication.tar
    untarred_folder = tar_name.split('.')[0] # publication
    untarred_path = os.path.join(path, untarred_folder) # data/raw/publication
    if not os.path.exists(untarred_path):
        if not os.path.exists(tar_path):
            print(f"downloading ${url}")
            try:
                with requests.get(url, stream=True) as response:
                    response.raise_for_status()
                    with open(tar_path, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            f.write(chunk)
            except requests.exceptions.RequestException as e:
                print("Error downloading the file:", e)

        print(f"untar ${tar_name}")
        with tarfile.open(tar_path, "r") as tar:
            tar.extractall(path)

        print('cleaning')
        os.remove(tar_path)


# Download data into /data/raw
for tar in openaire_files:
    download_and_extract(tar, "/app/openaire/data/raw")

## Import libraries

In [ ]:
import json

import glob
import pandas as pd

import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import StructType
from pyspark.sql import SparkSession
from IPython.display import JSON as pretty_print

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

## Load the datasets

If you try to load the data straight into memory, one `part file` would fit

In [ ]:
df = pd.read_json('data/raw/publication/part-00000-701d0814-12c8-4855-8340-65934f66e3f1-c000.json.gz', compression='gzip', lines=True)
df.head(2)

However, if you try to load the whole thing, even just the publications, the chances are slim.

If you try uncommenting and running the following lines, after some time, the kernel will die while trying and restart.

In [ ]:
# files = sorted(glob.glob('data/raw/publication/part-*.txt.gz'))
# publications_df = pd.concat(pd.read_json(f, compression='gzip', lines=True) for f in files)

So, let's see how `Spark` can help us.

First thing first, let's create the Spark session.

In [ ]:
spark = SparkSession.builder.getOrCreate()

Let's define now a few variables containing the schema followed by OpenAIRE data.

In [ ]:
resultSchema = '{"fields":[{"metadata":{},"name":"author","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"fullname","nullable":true,"type":"string"},{"metadata":{},"name":"name","nullable":true,"type":"string"},{"metadata":{},"name":"pid","nullable":true,"type":{"fields":[{"metadata":{},"name":"id","nullable":true,"type":{"fields":[{"metadata":{},"name":"scheme","nullable":true,"type":"string"},{"metadata":{},"name":"value","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"provenance","nullable":true,"type":{"fields":[{"metadata":{},"name":"provenance","nullable":true,"type":"string"},{"metadata":{},"name":"trust","nullable":true,"type":"string"}],"type":"struct"}}],"type":"struct"}},{"metadata":{},"name":"rank","nullable":true,"type":"long"},{"metadata":{},"name":"surname","nullable":true,"type":"string"}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"bestaccessright","nullable":true,"type":{"fields":[{"metadata":{},"name":"code","nullable":true,"type":"string"},{"metadata":{},"name":"label","nullable":true,"type":"string"},{"metadata":{},"name":"scheme","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"container","nullable":true,"type":{"fields":[{"metadata":{},"name":"conferencedate","nullable":true,"type":"string"},{"metadata":{},"name":"conferenceplace","nullable":true,"type":"string"},{"metadata":{},"name":"edition","nullable":true,"type":"string"},{"metadata":{},"name":"ep","nullable":true,"type":"string"},{"metadata":{},"name":"iss","nullable":true,"type":"string"},{"metadata":{},"name":"issnLinking","nullable":true,"type":"string"},{"metadata":{},"name":"issnOnline","nullable":true,"type":"string"},{"metadata":{},"name":"issnPrinted","nullable":true,"type":"string"},{"metadata":{},"name":"name","nullable":true,"type":"string"},{"metadata":{},"name":"sp","nullable":true,"type":"string"},{"metadata":{},"name":"vol","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"contributor","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"country","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"code","nullable":true,"type":"string"},{"metadata":{},"name":"label","nullable":true,"type":"string"},{"metadata":{},"name":"provenance","nullable":true,"type":{"fields":[{"metadata":{},"name":"provenance","nullable":true,"type":"string"},{"metadata":{},"name":"trust","nullable":true,"type":"string"}],"type":"struct"}}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"coverage","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"dateofcollection","nullable":true,"type":"string"},{"metadata":{},"name":"description","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"embargoenddate","nullable":true,"type":"string"},{"metadata":{},"name":"format","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"green","nullable":true,"type":"boolean"},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"inDiamondJournal","nullable":true,"type":"boolean"},{"metadata":{},"name":"indicators","nullable":true,"type":{"fields":[{"metadata":{},"name":"bipIndicators","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"class","nullable":true,"type":"string"},{"metadata":{},"name":"indicator","nullable":true,"type":"string"},{"metadata":{},"name":"score","nullable":true,"type":"string"}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"usageCounts","nullable":true,"type":{"fields":[{"metadata":{},"name":"downloads","nullable":true,"type":"string"},{"metadata":{},"name":"views","nullable":true,"type":"string"}],"type":"struct"}}],"type":"struct"}},{"metadata":{},"name":"instance","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"accessright","nullable":true,"type":{"fields":[{"metadata":{},"name":"code","nullable":true,"type":"string"},{"metadata":{},"name":"label","nullable":true,"type":"string"},{"metadata":{},"name":"openAccessRoute","nullable":true,"type":"string"},{"metadata":{},"name":"scheme","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"alternateIdentifier","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"scheme","nullable":true,"type":"string"},{"metadata":{},"name":"value","nullable":true,"type":"string"}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"articleprocessingcharge","nullable":true,"type":{"fields":[{"metadata":{},"name":"amount","nullable":true,"type":"string"},{"metadata":{},"name":"currency","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"license","nullable":true,"type":"string"},{"metadata":{},"name":"pid","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"scheme","nullable":true,"type":"string"},{"metadata":{},"name":"value","nullable":true,"type":"string"}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"publicationdate","nullable":true,"type":"string"},{"metadata":{},"name":"refereed","nullable":true,"type":"string"},{"metadata":{},"name":"type","nullable":true,"type":"string"},{"metadata":{},"name":"url","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"language","nullable":true,"type":{"fields":[{"metadata":{},"name":"code","nullable":true,"type":"string"},{"metadata":{},"name":"label","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"lastupdatetimestamp","nullable":true,"type":"long"},{"metadata":{},"name":"maintitle","nullable":true,"type":"string"},{"metadata":{},"name":"openAccessColor","nullable":true,"type":"string"},{"metadata":{},"name":"originalId","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"pid","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"scheme","nullable":true,"type":"string"},{"metadata":{},"name":"value","nullable":true,"type":"string"}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"publicationdate","nullable":true,"type":"string"},{"metadata":{},"name":"publiclyFunded","nullable":true,"type":"boolean"},{"metadata":{},"name":"publisher","nullable":true,"type":"string"},{"metadata":{},"name":"source","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"subjects","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"provenance","nullable":true,"type":{"fields":[{"metadata":{},"name":"provenance","nullable":true,"type":"string"},{"metadata":{},"name":"trust","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"subject","nullable":true,"type":{"fields":[{"metadata":{},"name":"scheme","nullable":true,"type":"string"},{"metadata":{},"name":"value","nullable":true,"type":"string"}],"type":"struct"}}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"subtitle","nullable":true,"type":"string"},{"metadata":{},"name":"type","nullable":true,"type":"string"}],"type":"struct"}'
datasourceSchema = '{"fields":[{"metadata":{},"name":"accessrights","nullable":true,"type":"string"},{"metadata":{},"name":"certificates","nullable":true,"type":"string"},{"metadata":{},"name":"citationguidelineurl","nullable":true,"type":"string"},{"metadata":{},"name":"databaseaccessrestriction","nullable":true,"type":"string"},{"metadata":{},"name":"datasourcetype","nullable":true,"type":{"fields":[{"metadata":{},"name":"scheme","nullable":true,"type":"string"},{"metadata":{},"name":"value","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"datauploadrestriction","nullable":true,"type":"string"},{"metadata":{},"name":"dateofvalidation","nullable":true,"type":"string"},{"metadata":{},"name":"description","nullable":true,"type":"string"},{"metadata":{},"name":"englishname","nullable":true,"type":"string"},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"journal","nullable":true,"type":{"fields":[{"metadata":{},"name":"issnLinking","nullable":true,"type":"string"},{"metadata":{},"name":"issnOnline","nullable":true,"type":"string"},{"metadata":{},"name":"issnPrinted","nullable":true,"type":"string"},{"metadata":{},"name":"name","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"languages","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"logourl","nullable":true,"type":"string"},{"metadata":{},"name":"missionstatementurl","nullable":true,"type":"string"},{"metadata":{},"name":"officialname","nullable":true,"type":"string"},{"metadata":{},"name":"openairecompatibility","nullable":true,"type":"string"},{"metadata":{},"name":"originalId","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"pid","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"scheme","nullable":true,"type":"string"},{"metadata":{},"name":"value","nullable":true,"type":"string"}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"pidsystems","nullable":true,"type":"string"},{"metadata":{},"name":"policies","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"releaseenddate","nullable":true,"type":"string"},{"metadata":{},"name":"releasestartdate","nullable":true,"type":"string"},{"metadata":{},"name":"subjects","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"uploadrights","nullable":true,"type":"string"},{"metadata":{},"name":"versioning","nullable":true,"type":"boolean"},{"metadata":{},"name":"websiteurl","nullable":true,"type":"string"}],"type":"struct"}'
organizationSchema = '{"fields":[{"metadata":{},"name":"alternativenames","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"country","nullable":true,"type":{"fields":[{"metadata":{},"name":"code","nullable":true,"type":"string"},{"metadata":{},"name":"label","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"legalname","nullable":true,"type":"string"},{"metadata":{},"name":"legalshortname","nullable":true,"type":"string"},{"metadata":{},"name":"pid","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"scheme","nullable":true,"type":"string"},{"metadata":{},"name":"value","nullable":true,"type":"string"}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"websiteurl","nullable":true,"type":"string"}],"type":"struct"}'
projectSchema = '{"fields":[{"metadata":{},"name":"acronym","nullable":true,"type":"string"},{"metadata":{},"name":"callidentifier","nullable":true,"type":"string"},{"metadata":{},"name":"code","nullable":true,"type":"string"},{"metadata":{},"name":"enddate","nullable":true,"type":"string"},{"metadata":{},"name":"funding","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"funding_stream","nullable":true,"type":{"fields":[{"metadata":{},"name":"description","nullable":true,"type":"string"},{"metadata":{},"name":"id","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"jurisdiction","nullable":true,"type":"string"},{"metadata":{},"name":"name","nullable":true,"type":"string"},{"metadata":{},"name":"shortName","nullable":true,"type":"string"}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"granted","nullable":true,"type":{"fields":[{"metadata":{},"name":"currency","nullable":true,"type":"string"},{"metadata":{},"name":"fundedamount","nullable":true,"type":"double"},{"metadata":{},"name":"totalcost","nullable":true,"type":"double"}],"type":"struct"}},{"metadata":{},"name":"h2020programme","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"code","nullable":true,"type":"string"},{"metadata":{},"name":"description","nullable":true,"type":"string"}],"type":"struct"},"type":"array"}},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"keywords","nullable":true,"type":"string"},{"metadata":{},"name":"openaccessmandatefordataset","nullable":true,"type":"boolean"},{"metadata":{},"name":"openaccessmandateforpublications","nullable":true,"type":"boolean"},{"metadata":{},"name":"startdate","nullable":true,"type":"string"},{"metadata":{},"name":"subject","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"summary","nullable":true,"type":"string"},{"metadata":{},"name":"title","nullable":true,"type":"string"},{"metadata":{},"name":"websiteurl","nullable":true,"type":"string"}],"type":"struct"}'
communitySchema = '{"fields":[{"metadata":{},"name":"acronym","nullable":true,"type":"string"},{"metadata":{},"name":"description","nullable":true,"type":"string"},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"subject","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"type","nullable":true,"type":"string"},{"metadata":{},"name":"zenodo_community","nullable":true,"type":"string"}],"type":"struct"}'
relationSchema = '{"fields":[{"metadata":{},"name":"provenance","nullable":true,"type":{"fields":[{"metadata":{},"name":"provenance","nullable":true,"type":"string"},{"metadata":{},"name":"trust","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"reltype","nullable":true,"type":{"fields":[{"metadata":{},"name":"name","nullable":true,"type":"string"},{"metadata":{},"name":"type","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"source","nullable":true,"type":"string"},{"metadata":{},"name":"sourceType","nullable":true,"type":"string"},{"metadata":{},"name":"target","nullable":true,"type":"string"},{"metadata":{},"name":"targetType","nullable":true,"type":"string"},{"metadata":{},"name":"validated","nullable":true,"type":"boolean"},{"metadata":{},"name":"validationDate","nullable":true,"type":"string"}],"type":"struct"}'

Now, let's read the datasets about OpenAIRE entitities.

In [ ]:
inputPath = 'data/raw/'
 
publications = spark.read.schema(StructType.fromJson(json.loads(resultSchema))).json(inputPath + 'publication')
datasets = spark.read.schema(StructType.fromJson(json.loads(resultSchema))).json(inputPath + 'dataset')
softwares = spark.read.schema(StructType.fromJson(json.loads(resultSchema))).json(inputPath + 'software')
others = spark.read.schema(StructType.fromJson(json.loads(resultSchema))).json(inputPath + 'otherresearchproduct')
results = publications.unionByName(datasets).unionByName(softwares).unionByName(others)
datasources = spark.read.schema(StructType.fromJson(json.loads(datasourceSchema))).json(inputPath + 'datasource')
organizations = spark.read.schema(StructType.fromJson(json.loads(organizationSchema))).json(inputPath + 'organization')
projects = spark.read.schema(StructType.fromJson(json.loads(projectSchema))).json(inputPath + 'project')
communities = spark.read.schema(StructType.fromJson(json.loads(communitySchema))).json(inputPath + 'communities_infrastructures')
relations = spark.read.schema(StructType.fromJson(json.loads(relationSchema))).json(inputPath + 'relation')

Let's create some `Temporary views`, which is similar to a real SQL table that you can query via Spark.

In [ ]:
publications.createOrReplaceTempView("publications")
datasets.createOrReplaceTempView("datasets")
softwares.createOrReplaceTempView("software")
others.createOrReplaceTempView("others")
results.createOrReplaceTempView("results")
datasources.createOrReplaceTempView("datasources")
organizations.createOrReplaceTempView("organizations")
projects.createOrReplaceTempView("projects")
communities.createOrReplaceTempView("communities")
relations.createOrReplaceTempView("relations")


Ok, let's count the number of rows.

In [ ]:
print("number of publications %s"%publications.count())
print("number of datasets %s"%datasets.count())
print("number of software %s"%softwares.count())
print("number of other research products %s"%others.count())
print("number of results %s"%results.count())
print("number of datasources %s"%datasources.count())
print("number of organizations %s"%organizations.count())
print("number of communities %s"%communities.count())
print("number of projects %s"%projects.count())
print("number of relations %s"%relations.count())

By the way, the same could be achieved in SQL via Spark.

In [ ]:
spark.sql("SELECT COUNT(*) FROM publications").toPandas()

Let's show some data now. 
For example, a generic publication (link to documentation: https://graph.openaire.eu/docs/data-model/entities/result)

In [ ]:
pretty_print(json.loads(publications
                        .where("id='doi_dedup___::e87624cec4ec3ce1c247c89d69624a5b'")
                        .toJSON()
                        .first()), expanded=False)

Or a data source (link to documentation: https://graph.openaire.eu/docs/data-model/entities/data-source)

In [ ]:
pretty_print(json.loads(datasources
                        .where("id='fairsharing_::c3a690be93aa602ee2dc0ccab5b7b67e'")
                        .toJSON()
                        .first()), expanded=False)

An organization (link to documentation: https://graph.openaire.eu/docs/data-model/entities/organization)

In [ ]:
pretty_print(json.loads(organizations
                        .where("id='openorgs____::5836463160e0e5d1cd12997f7d2f0257'")
                        .toJSON()
                        .first()), expanded=False)

A project (link to documentation: https://graph.openaire.eu/docs/data-model/entities/project)

In [ ]:
pretty_print(json.loads(projects
                        .toJSON()
                        .first()), expanded=False)

A community (link to documentation: https://graph.openaire.eu/docs/data-model/entities/community)

In [ ]:
pretty_print(json.loads(communities
                        .where("acronym='mes'")
                        .toJSON()
                        .first()), expanded=False)

And finally, a relation (link to documentation: https://graph.openaire.eu/docs/data-model/relationships)

In [ ]:
pretty_print(json.loads(relations.toJSON().first()), expanded=False)

## Exercises 

All the exercises follow the template below.
```python
query = """
SELECT <columns>
FROM <table>
"""

spark.sql(query).limit(20).toPandas()
```

`toPandas()` results in the collection of all records in the PySpark DataFrame to the driver program and should be done only on a small subset of the data. running on larger dataset’s results in memory error and crashes the application.
`limit(20)` is added exactly for this purpose.

<div class="alert alert-info">
Group relations based on their semantics and count them; sort results in descending order, limit to the first 20. 
</div>

In [ ]:
query = """
SELECT reltype.name, COUNT(*) AS count 
FROM relations 
GROUP BY reltype.name 
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show information about publishing venues.
</div>

In [ ]:
query = """
SELECT container.issnLinking, container.issnOnline, container.issnPrinted, container.name 
FROM publications 
WHERE container IS NOT NULL
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Count and sort publications by citations.
</div>

In [ ]:
query = """
SELECT publications.id, pid.value, COUNT(*) AS count
FROM publications JOIN relations ON publications.id = relations.source 
WHERE reltype.name = 'IsCitedBy'
GROUP BY publications.id, pid.value
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

Since relations are bi-directional, so we could have used the dual semantic `Cites` and join on target ids. We would get the same results.


In [ ]:
query = """
SELECT publications.id, pid.value, COUNT(*) AS count
FROM publications JOIN relations ON publications.id = relations.target
WHERE reltype.name = 'Cites'
GROUP BY publications.id, pid.value
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the most occurring publication subjects; sort results in descending order; limit to the first 20.
</div>

In [ ]:
query = """
WITH terms AS (
    SELECT explode(subjects.subject.value) AS `term`
    FROM publications
)
SELECT term AS `subject term`,
        COUNT(*) AS count 
FROM terms 
GROUP BY term 
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the journals with the highest number of publications
</div>

In [ ]:
query = """
WITH journals AS (
    SELECT container.*
    FROM publications
    WHERE container IS NOT NULL
)
SELECT name, count(*) AS count 
FROM journals 
GROUP BY name 
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the number of projects per organization; sort results in descending order; limit to the first 20.

Hint: the `COALESCE` function can be oh help to select over the possible name forms of an organisation (e.g., short and full name). You can specify multiple columns to select and it will return the first column that is not null. 
</div>

In [ ]:
query = """
SELECT COALESCE(legalshortname, legalname) AS name, 
        COUNT(*) AS count 
FROM organizations JOIN relations ON organizations.id = relations.source
                                        AND reltype.name = 'isParticipant'
GROUP BY name 
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show projects with the highest number of associated results. 

Note: An `unidentified` project title is a placeholder for all the associations to a funder without knowing the specific project. It should be removed from the count.
</div>

In [ ]:
query = """
SELECT funding.shortName, code, title, COUNT(*) AS count 
FROM projects JOIN relations ON projects.id = relations.source
                                AND reltype.name = 'produces' 
                                AND not projects.title ilike '%unidentified%' 
GROUP BY funding.shortName, code, title
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

Strings can be manipulated as well on the fly

In [ ]:
query = """
SELECT CONCAT_WS(' / ',
                IF(SIZE(funding.shortName) > 0, ARRAY_JOIN(funding.shortName, ',', '-'), '?'), 
                COALESCE(code, '?'), 
                SUBSTRING(title, 0, 50)) AS project, COUNT(*) AS count 
FROM projects JOIN relations ON projects.id = relations.source 
                                AND reltype.name = 'produces' 
                                AND NOT projects.title ilike '%unidentified%' 
GROUP BY project 
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the most co-occurring publication subjects from controlled vocabularies (i.e., scheme != 'keyword') avoiding repetition; limit to the first 20.
</div>

In [ ]:
query = """
WITH subjects AS (
    WITH exploded_subjects (
        SELECT id, EXPLODE(subjects.subject) AS subject 
        FROM publications) 
    SELECT id, subject.value AS `subject` 
    FROM exploded_subjects 
    WHERE subject.scheme != 'keyword'
)
SELECT l.subject AS left, 
       r.subject AS right, 
       COUNT(*) AS count
FROM subjects AS l JOIN subjects AS r ON l.id = r.id AND l.subject < r.subject
GROUP BY left, right
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the number of research products per organization; sort results in descending order; limit to the first 20. The relation used is the affiliation, since in our data this relation links products to organization and not authors to organizations.
</div>

Organization short names can be empty, so the legal name could be a fallback option to use in `COALESCE`.

In [ ]:
query = """
SELECT legalshortname, legalname
FROM organizations 
WHERE legalshortname IS NULL and legalname IS NOT NULL
"""

spark.sql(query).limit(20).toPandas()

In [ ]:
query = """
SELECT COALESCE(legalshortname, legalname) AS organization,
        COUNT(*) AS count 
FROM organizations JOIN relations ON organizations.id = relations.source
                                        AND reltype.name = 'isAuthorInstitutionOf' 
GROUP BY organization
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the number of research products (per type) per organization.
</div>

In [ ]:
query = """
SELECT COALESCE(legalshortname, legalname) AS organization, 
       COUNT(*) AS total,
       COUNT(IF(type = 'publication', 1, NULL)) AS publication,
       COUNT(IF(type = 'dataset', 1, NULL)) AS dataset,
       COUNT(IF(type = 'software', 1, NULL)) AS software,
       COUNT(IF(type = 'other', 1, NULL)) AS other
FROM results JOIN organizations JOIN relations ON organizations.id = relations.source
                                                 AND results.id = relations.target 
                                                 AND reltype.name = 'isAuthorInstitutionOf' 
GROUP BY organization 
ORDER BY total DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show result access types per organization.
</div>

In [ ]:
query = """
SELECT COALESCE(legalshortname, legalname) AS organization, 
       COUNT(*) as total,
       COUNT(IF(bestaccessright.label = 'OPEN', 1, NULL)) AS open,
       COUNT(IF(bestaccessright.label = 'EMBARGO', 1, NULL)) AS embargo,
       COUNT(IF(bestaccessright.label = 'CLOSED', 1, NULL)) AS closed
FROM organizations JOIN relations JOIN results ON organizations.id = relations.source 
                                                 AND results.id = relations.target 
                                                 AND reltype.name = 'isAuthorInstitutionOf'
GROUP BY organization
ORDER BY total DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the result access types per country of the organizations.
</div>

In [ ]:
query = """
SELECT organizations.country.code AS country, 
       COUNT(*) AS total,
       COUNT(IF(bestaccessright.label = 'OPEN', 1, NULL)) AS open,
       COUNT(IF(bestaccessright.label = 'EMBARGO', 1, NULL)) AS embargo,
       COUNT(IF(bestaccessright.label = 'CLOSED', 1, NULL)) AS closed
FROM organizations JOIN relations JOIN results ON organizations.id = relations.source
                                                 AND results.id = relations.target 
                                                 AND reltype.name = 'isAuthorInstitutionOf'
WHERE organizations.country IS NOT NULL
GROUP BY organizations.country.code
ORDER BY total DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the collaboration network of countries participating in projects with respect to the partecipating organizations.
</div>

In [ ]:
query = """
WITH countryProject AS (
    SELECT country.code AS country, 
           target AS id 
    FROM organizations JOIN relations ON reltype.name = 'isParticipant' AND source = organizations.id
    WHERE country IS NOT NULL
)
SELECT l.country AS left, 
       r.country AS right,
       COUNT(*) AS count 
FROM countryProject AS l JOIN countryProject AS r ON l.id = r.id AND l.country <= r.country
GROUP BY left, right 
ORDER BY count DESC
"""

edges = spark.sql(query).toPandas()
edges

Results can be modeled as a graph and analysed. Let's try doing so with igraph and feed it with country couples and the number of coparticipated projects.

In [ ]:
import igraph as ig

G = ig.Graph.TupleList(
    edges=edges[['left', 'right', 'count']].values,
    vertex_name_attr='countrycode',
    edge_attrs = ['weight'],
    directed=False)

Let's find the number of nodes in the graph.

In [ ]:
G.vcount()

Now, let's find the number of edges.

In [ ]:
G.ecount()

A generic node can be inspected like so

In [ ]:
G.vs[0]

Instead, a generic edge can be inspected with

In [ ]:
G.es[0]

Ok. Let's try to plot something. The whole network looks like this.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(10, 10))
max_w = np.max(G.es['weight'])
ig.plot(G, vertex_label=G.vs['countrycode'], vertex_size=10, edge_width=2, edge_color='gray', target=ax)

Now, let's filter by country, say Italy (i.e., IT)

In [ ]:
H = G.induced_subgraph(G.neighborhood(G.vs.find(countrycode_eq = 'IT')))
H.summary()

In [ ]:
H.vs['color'] = 'blue'
H.vs.find(countrycode_eq = 'IT')['color'] = 'red'
fig, ax = plt.subplots(figsize=(10, 10))
ig.plot(H, vertex_label=H.vs['countrycode'], vertex_size=30, edge_width=2, edge_color='gray',target=ax)

Still a lot of collaborations there. Let's plot a country with less collaborations, say Maldives (i.e., MV).

In [ ]:
H = G.induced_subgraph(G.neighborhood(G.vs.find(countrycode_eq = 'MV'))) # Maldives
H.summary()

In [ ]:
H.vs['color'] = 'blue'
H.vs.find(countrycode_eq = 'MV')['color'] = 'red'
fig, ax = plt.subplots(figsize=(10, 10))
ig.plot(H, vertex_label=H.vs['countrycode'], vertex_size=30, edge_width=2, edge_color='gray',target=ax)

<div class="alert alert-info">
Show international project collaborations; focus on organizations.
</div>

In [ ]:
query = """
WITH countryProject AS (
    SELECT country.code AS country, 
           target AS id 
    FROM organizations JOIN relations ON reltype.name = 'isParticipant' 
                                          AND source = organizations.id
    WHERE country IS NOT NULL
)
SELECT l.country AS left, 
       r.country AS right, 
       COUNT(*) AS count 
FROM countryProject AS l JOIN countryProject AS r ON l.id = r.id AND l.country < r.country
GROUP BY left, right 
ORDER BY count DESC
"""

spark.sql(query).toPandas() 

<div class="alert alert-info">
Show the organisations collaborating in projects more often.
</div>

In [ ]:
query = """
WITH orgProject AS (
    SELECT COALESCE(legalshortname, legalname) AS organization, 
           target AS id 
    FROM organizations JOIN relations ON reltype.name = 'isParticipant' 
                                        AND source = organizations.id
)
SELECT l.organization AS left,
       r.organization AS right,
       COUNT(*) AS count
FROM orgProject AS l JOIN orgProject AS r ON l.id = r.id AND l.organization < r.organization
GROUP BY left, right 
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the organizations co-authoring papers more often.
</div>

In [ ]:
query = """
WITH orgProduct AS (
    SELECT COALESCE(legalshortname, legalname) AS organization, 
           target AS id 
    FROM organizations JOIN relations ON reltype.name = 'isAuthorInstitutionOf' 
                                          AND source = organizations.id
)
SELECT l.organization AS left, 
       r.organization AS right,
       COUNT(*) AS count 
FROM orgProduct AS l JOIN orgProduct AS r ON l.id = r.id AND l.organization < r.organization
GROUP BY left, right 
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Summarize the access rights over the years.
</div>

In [ ]:
query = """
SELECT bestaccessright.label AS accessright,
       SUBSTRING(publicationdate, 0,4) AS year,
       COUNT(*) AS count
FROM results
WHERE bestaccessright IS NOT NULL AND publicationdate IS NOT NULL
GROUP BY accessright, year
ORDER BY count DESC
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the number of publications supplemented by datasets.
</div>

In [ ]:
query = """
SELECT COUNT(*) AS count
FROM relations JOIN publications JOIN datasets ON reltype.name = 'IsSupplementedBy' 
                                        AND publications.id = relations.source 
                                        AND datasets.id = relations.target
"""

spark.sql(query).limit(20).toPandas()

<div class="alert alert-info">
Show the number of peer reviewed publications with doi split by openAccessColor and the availability of a green deposition
</div>

In [ ]:
query = """

SELECT COUNT(id), green , openAccessColor
FROM publications
WHERE ARRAY_CONTAINS(instance.refereed, 'peerReviewed') 
AND  ARRAY_CONTAINS(pid.scheme, 'doi')
AND green IS NOT NULL AND openAccessColor IS NOT NULL
GROUP BY openAccessColor, green

"""
spark.sql(query).limit(20).toPandas()